In [1]:
%matplotlib widget

In [2]:
import numpy as np
import xarray as xr

from scipy.spatial import Delaunay
from scipy.spatial.transform import Rotation as rot

# plotting
import matplotlib.pyplot as plt
import pims
import ipyvolume as ipv

# interactive plots
import ipywidgets as widgets
from ipywidgets import VBox, HBox, IntSlider, interactive_output, Label, Layout
from IPython.display import display


import libo
from libo.io import tc3

import basic_meshes as bm
from experimental_setup import get_setup

import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vs

# Setup

In [3]:
setup = get_setup(5)

profile_raster_width = 4
trace_raster_width = 4

# Import and process experimental data

### Temperature data

In [4]:
dsx_sensor_data = tc3.to_xarray(tc3.read_db(setup.measurement_data_id))
dsx_temperature_data = dsx_sensor_data[["MH24_T01", "MH24_T02", "MH24_T03"]].dropna("time")

### Machine data

In [5]:
# import machine data
dsx_machine_data = tc3.to_xarray(tc3.read_db(setup.system_data_index))


# collect relevant data
relevant_data_names = ["FB_X", "FB_Y", "FB_Z", "FB_Rx", "FB_Ry", "FB_Rz", "trigSchweissen", "trigScan2_prog"]
dsx_relevant_machine_data = dsx_machine_data[relevant_data_names].dropna("time")
interpolated_temperature_data = dsx_temperature_data.interp_like(dsx_relevant_machine_data)
dsx_relevant_machine_data = xr.merge([interpolated_temperature_data, dsx_relevant_machine_data])


# extract system movement data
dsx_machine_position_data = dsx_relevant_machine_data[["FB_X", "FB_Y", "FB_Z", "FB_Rx", "FB_Ry", "FB_Rz"]]
machine_coordinates_base = dsx_machine_position_data.to_array().data[:3, :]
machine_angles_degree_base = dsx_machine_position_data.to_array().data[3:6, :]

### Scanner data

In [6]:
num_scans = len(setup.scan_data_scan_idx)
list_dsx_scan_data = []
for i in range(num_scans):
    data = libo.io.tools.merge_scan_tcp(scan=setup.scan_data_scan_idx[i], tcp=setup.scan_data_tcp_idx[i])
    interpolated_temperature_data = dsx_temperature_data.interp(time=data.time)
    list_dsx_scan_data += [xr.merge([interpolated_temperature_data, data])]

# Define coordinate systems


### Fixed systems
Important note: Measured data is not perfectly orthogonal. We use the cross product to get a z axis which is perfectly orthogonal (in floating point precision) and recalculate a perfectly orthogonal y-axis. There are certainly better ways to address this problem.

In [7]:
# Calculate orthogonal z axis and construct reference coordinate system from x and z axis
vec_x = setup.offset_ox_ref - setup.origin_ref
vec_y = setup.offset_oy_ref - setup.origin_ref
vec_z = np.cross(vec_x, vec_y)

cs_ref_in_base = tf.LocalCoordinateSystem.construct_from_xz_and_orientation(vec_x, vec_z, origin=setup.origin_ref)

cs_sp_in_ref = setup.cs_sp_in_ref
cs_torch_tcp_in_flange = setup.cs_torch_tcp_in_flange
cs_scanner_tcp_in_flange = setup.cs_scanner_tcp_in_flange
cs_cam_in_flange = setup.cs_cam_in_flange
cs_scan_in_scanner_tcp = setup.cs_scan_in_scanner_tcp
list_cs_temp_in_ref = setup.list_cs_temp_in_ref

### Derived systems

In [8]:
cs_sp_in_base = cs_sp_in_ref + cs_ref_in_base
cs_scanner_tcp_in_torch_tcp = cs_scanner_tcp_in_flange - cs_torch_tcp_in_flange
cs_cam_in_torch_tcp = cs_cam_in_flange - cs_torch_tcp_in_flange
cs_torch_tcp_in_scanner_tcp = cs_torch_tcp_in_flange - cs_scanner_tcp_in_flange
cs_cam_in_scanner_tcp = cs_cam_in_flange - cs_scanner_tcp_in_flange
list_cs_temp_in_sp = []
for i in range(len(list_cs_temp_in_ref)):
    list_cs_temp_in_sp += [list_cs_temp_in_ref[i] - cs_sp_in_ref]

### Functions for variable systems

In [9]:
def get_cs_scanner_tcp_in_sp(scan_data, index):
    angles = scan_data.scan_tcp.sel(tcp_variable=["Rx", "Ry", "Rz"]).data[index]
    orientation = rot.from_euler(angles=angles, seq="xyz", degrees=True).as_matrix()
    coordinates = scan_data.scan_tcp.sel(tcp_variable=["X", "Y", "Z"]).data[index]
    cs_scanner_tcp_in_base = tf.LocalCoordinateSystem(basis=orientation, origin=coordinates)
    return cs_scanner_tcp_in_base - cs_ref_in_base - cs_sp_in_ref


def get_cs_scan_in_sp(scan_data, index):
    return cs_scan_in_scanner_tcp + get_cs_scanner_tcp_in_sp(scan_data, index)

# Define data transformations

In [10]:
def coordinates_to_child_from_parent(coordinates, cs_child):
    rotation = cs_child.orientation.transpose()
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates - translation)


def coordinates_to_parent_from_child(coordinates, cs_child):
    rotation = cs_child.orientation
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates) + translation

# Transform data to specimen coordinate system

In [11]:
def transform_scan_data_to_sp(scan_data_in_scan):
    scanned_profiles_in_sp = []
    num_scans = scan_data_in_scan.profile.size
    for i in range(num_scans):
        scanned_profile_in_scan = scan_data_in_scan.scan_line.data[i].transpose()
        scanned_profiles_in_sp += [
            coordinates_to_parent_from_child(scanned_profile_in_scan, get_cs_scan_in_sp(scan_data_in_scan, i))
        ]
    return np.array(scanned_profiles_in_sp, float)


scanned_profiles_layer_0_sp = transform_scan_data_to_sp(list_dsx_scan_data[0])
scanned_profiles_layer_1_sp = transform_scan_data_to_sp(list_dsx_scan_data[1])

# Create theoretical geometry

In [12]:
# create points
pt_0 = [150, 8]
pt_1_1 = [np.tan(setup.groove_angle_start / 360 * np.pi) * 8, 8]
pt_1_2 = [np.tan(setup.groove_angle_end / 360 * np.pi) * 8, 8]
pt_2 = [0, 0]
pt_3 = [150, 0]

# create shapes
shape_p1_r = geo.Shape().add_line_segments([pt_0, pt_1_1, pt_2])
shape_p2_r = geo.Shape().add_line_segments([pt_0, pt_1_2, pt_2])
shape_p1_l = shape_p1_r.reflect([1, 0])
shape_p2_l = shape_p2_r.reflect([1, 0])

# create profiles
profile_1 = geo.Profile([shape_p1_l, shape_p1_r])
profile_2 = geo.Profile([shape_p2_l, shape_p2_r])

# create variable profile
variable_profile = geo.VariableProfile([profile_1, profile_2], [0, 1], [geo.linear_profile_interpolation_sbs])

# create trace
trace = geo.Trace(geo.LinearHorizontalTraceSegment(350))

# create geometry
geometry = geo.Geometry(variable_profile, trace)

# rasterize profiles
profile_1_data = profile_1.rasterize(4)
profile_2_data = profile_2.rasterize(4)

# rasterize geometry
geometry_data_sp = geometry.rasterize(profile_raster_width=profile_raster_width, trace_raster_width=trace_raster_width)

# calculate triangles
triangles = Delaunay(geometry_data_sp.transpose()[:, :2]).simplices

# transform data to specimen coordinate system
geometry_data_sp = np.matmul(tf.rotation_matrix_x(-np.pi / 2), geometry_data_sp)

# Create sequence

In [13]:
# create time line

num_timesteps_scan_0 = 400
num_scans_layer_0 = list_dsx_scan_data[0].profile.size
indices_scan_0 = np.arange(0, num_scans_layer_0 + 0.5, num_scans_layer_0 / num_timesteps_scan_0)
indices_scan_0 = np.array(np.round(indices_scan_0), int)


num_timesteps_movement = 800
idx_scan_1_start = np.argmax(dsx_relevant_machine_data.trigScan2_prog.data > 0.1)
num_movement_data = idx_scan_1_start
indices_movement = np.arange(0, num_movement_data + 0.5, num_movement_data / num_timesteps_movement)
indices_movement = np.array(np.round(indices_movement), int)
num_shown_video_frames = (dsx_relevant_machine_data.trigSchweissen.data[indices_movement] > 0).sum()

num_timesteps_scan_1 = 400
num_scans_layer_1 = list_dsx_scan_data[1].profile.size
indices_scan_1 = np.arange(0, num_scans_layer_1 + 0.5, num_scans_layer_1 / num_timesteps_scan_1)
indices_scan_1 = np.array(np.round(indices_scan_1), int)


num_timesteps_total = num_timesteps_scan_0 + num_timesteps_movement + num_timesteps_scan_1

# Extract frames

In [14]:
# v = pims.PyAVReaderIndexed("./hd_05.avi")
# v

In [15]:
video_frames = np.array(np.load("./hd_05.frames.npy"))
num_video_frames = video_frames.shape[0]
delta_frame = num_video_frames / num_shown_video_frames
indices_frames = np.arange(0, num_video_frames - 0.5, delta_frame)
# video_data_array = np.asarray(v[100][::3,::3])
indices_frames = np.array(np.round(indices_frames), int)

video_frames = video_frames[indices_frames]

# Create meshes

In [16]:
scale_scanner = 15


[vert_scanner, tri_scanner] = bm.create_cone_mesh(40)
vert_scanner = vert_scanner + np.array([0, 0, -1])[:, np.newaxis]
vert_scanner = np.matmul([[scale_scanner, 0, 0], [0, scale_scanner, 0], [0, 0, scale_scanner * 2]], vert_scanner)


scale_torch = 15

[vert_torch, tri_torch] = bm.create_cone_mesh(40)
vert_torch = vert_torch + np.array([0, 0, -1])[:, np.newaxis]
vert_torch = np.matmul([[scale_torch, 0, 0], [0, scale_torch, 0], [0, 0, scale_torch * 2]], vert_torch)


scale_camera = 15

[vert_camera, tri_camera] = bm.create_cone_mesh(40)
vert_camera = vert_camera + np.array([0, 0, -1])[:, np.newaxis]
vert_camera = np.matmul([[scale_camera, 0, 0], [0, scale_camera, 0], [0, 0, scale_camera * 2]], vert_camera)
vert_camera = vert_camera + np.array([0, 0, 200])[:, np.newaxis]

cylinder_radius = 5
cylinder_height = 1
[vertices_cylinder, triangle_indices_cylinder] = bm.create_unit_cylinder_mesh(40, 1)
vertices_cylinder = np.matmul(
    [[cylinder_radius, 0, 0], [0, cylinder_radius, 0], [0, 0, cylinder_height]], vertices_cylinder
)

# Create weld mesh

In [17]:
scp = scanned_profiles_layer_1_sp[::5, :, ::5]

# join all points
scp = np.swapaxes(scp, 1, 2)
scp = np.reshape(scp, (scp.shape[0] * scp.shape[1], 3))

# sort point by x-value
scp = scp[scp[:, 2] < 5]
scp = scp[scp[:, 2] > -5]
scp = scp[scp[:, 1] > -1]
idx_sort = np.argsort(scp[:, 0])
data_weld = scp[idx_sort]

# triangulate
triangles_weld = Delaunay(data_weld[:, [0, 2]]).simplices
max_id = np.max(triangles_weld, axis=1)
idx_sort = np.argsort(max_id)

triangles_weld = triangles_weld[idx_sort]
num_triangles_weld = triangles_weld.shape[0]
num_points_weld = data_weld.shape[0]

In [18]:
out2 = widgets.Output(layout={"border": "0px solid black"})
# create figure inside output widget
with out2:
    fig = plt.figure()
    fig.canvas.layout.height = "600px"
    fig.canvas.layout.width = "800px"
    gs = fig.add_gridspec(1, 1)
    ax_0 = fig.add_subplot(gs[0, 0])


ipvfig = ipv.figure(width=800, height=600, lighting=True)


# ipv.show()

play = widgets.Play(
    value=0, min=0, max=num_timesteps_total, step=1, interval=10, description="Press play", disabled=False
)
time_slider = IntSlider(min=0, max=num_timesteps_total, step=1, description="time", continuous_update=True)
w1 = dict(time=time_slider)

widgets.jslink((play, "value"), (time_slider, "value"))
w = {**w1}

l_t0 = Label("Thermoelement 1: ")
l_t1 = Label("Thermoelement 2: ")
l_t2 = Label("Thermoelement 3: ")


upper_box = HBox()
lower_box = HBox()
info_box = VBox([l_t0, l_t1, l_t2], layout=Layout(top="200px", width="200px"))
box = VBox([upper_box, lower_box])

ipv.plot_trisurf(
    geometry_data_sp[0], geometry_data_sp[1], geometry_data_sp[2], triangles=triangles, color=[0.9, 0.9, 0.9]
)
ipv.plot_trisurf(vert_scanner[0], vert_scanner[1], vert_scanner[2], triangles=tri_scanner, color=[1, 0, 0])
ipv.plot_trisurf(vert_torch[0], vert_torch[1], vert_torch[2], triangles=tri_torch, color=[0, 1, 0])
ipv.plot_trisurf(
    vertices_cylinder[0],
    vertices_cylinder[1],
    vertices_cylinder[2],
    triangles=triangle_indices_cylinder,
    color=[0, 0, 1],
)
ipv.plot_trisurf(
    vertices_cylinder[0],
    vertices_cylinder[1],
    vertices_cylinder[2],
    triangles=triangle_indices_cylinder,
    color=[0, 0, 1],
)
ipv.plot_trisurf(
    vertices_cylinder[0],
    vertices_cylinder[1],
    vertices_cylinder[2],
    triangles=triangle_indices_cylinder,
    color=[0, 0, 1],
)
ipv.plot_trisurf(vert_camera[0], vert_camera[1], vert_camera[2], triangles=tri_camera, color=[1, 1, 0])
scanned_profiles = np.concatenate((scanned_profiles_layer_0_sp, scanned_profiles_layer_1_sp), axis=0)
ipv.scatter(
    scanned_profiles[:, 0, :],
    scanned_profiles[:, 1, :],
    scanned_profiles[:, 2, :],
    size=1,
    marker="sphere",
    color="red",
)
ipv.plot_trisurf(data_weld[:, 0], data_weld[:, 1], data_weld[:, 2], triangles=triangles_weld, color=[1, 1, 0])

x_min = 0
x_max = 350
z_min = -175
z_max = 175
y_min = -20
y_max = 330
ipv.xlim(x_min, x_max)
ipv.ylim(y_min, y_max)
ipv.zlim(z_min, z_max)

d_x = x_max - x_min
d_y = y_max - y_min
d_z = z_max - z_min

s_x = d_x / d_y
s_y = d_y / d_y
s_z = d_z / d_y

axix_correction_scale_mat = [[s_x, 0, 0], [0, s_y, 0], [0, 0, s_z]]


cs_scanner_schematic = tf.LocalCoordinateSystem(origin=[0, 0, -260])

idx_switch_cs = np.argmax(dsx_machine_position_data.FB_Rz.dropna("time").diff("time").data)
idx_scan_1 = np.argmax(dsx_relevant_machine_data.trigScan2_prog.data > 0.1)


def temperature_to_color(temperature):
    temperature = np.clip(temperature, 0, 600)
    if temperature < 300:
        weight = temperature / 300
        return [0 + weight, 1, 0]
    else:
        weight = (temperature - 300) / 300
        return [1, 1 - weight, 0]


def transform_and_update_mesh(vertex_data_mesh, cs_transformation, ipv_idx, scale_x=1, scale_y=1, scale_z=1):
    scale_mat = [[scale_x, 0, 0], [0, scale_y, 0], [0, 0, scale_z]]
    vertex_data_trans = np.matmul(scale_mat, vertex_data_mesh)
    vertex_data_trans = np.matmul(cs_transformation.orientation, vertex_data_trans)
    vertex_data_trans = np.matmul(axix_correction_scale_mat, vertex_data_trans)
    vertex_data_trans = vertex_data_trans + cs_transformation.location[:, np.newaxis]
    ipvfig.meshes[ipv_idx].x = vertex_data_trans[0]
    ipvfig.meshes[ipv_idx].y = vertex_data_trans[1]
    ipvfig.meshes[ipv_idx].z = vertex_data_trans[2]


def update_meshes(cs_scanner_in_sp, cs_torch_in_sp, cs_cam_in_sp, torch_color, t_0, t_1, t_2):
    transform_and_update_mesh(vert_scanner, cs_scanner_in_sp, 1)
    transform_and_update_mesh(vert_torch, cs_torch_in_sp, 2)
    transform_and_update_mesh(vert_camera, cs_cam_in_sp, 6)
    ipvfig.meshes[2].color = torch_color

    transform_and_update_mesh(vertices_cylinder, list_cs_temp_in_sp[0], 3, scale_z=t_0 / 2)
    transform_and_update_mesh(vertices_cylinder, list_cs_temp_in_sp[1], 4, scale_z=t_1 / 2)
    transform_and_update_mesh(vertices_cylinder, list_cs_temp_in_sp[2], 5, scale_z=t_2 / 2)

    ipvfig.meshes[3].color = temperature_to_color(t_0)
    ipvfig.meshes[4].color = temperature_to_color(t_1)
    ipvfig.meshes[5].color = temperature_to_color(t_2)
    l_t0.value = "Thermoelement 1: " + str(np.round(t_0, decimals=1)) + " °C"
    l_t1.value = "Thermoelement 2: " + str(np.round(t_1, decimals=1)) + " °C"
    l_t2.value = "Thermoelement 3: " + str(np.round(t_2, decimals=1)) + " °C"


def visualize_scan_phase(scan_data, scanned_profiles_in_sp, idx, num_timesteps_scan, idx_offset=0):
    cs_scanner_tcp_in_sp = get_cs_scanner_tcp_in_sp(scan_data, idx)
    cs_scanner_in_sp = cs_scanner_schematic + cs_scanner_tcp_in_sp
    cs_torch_in_sp = cs_torch_tcp_in_scanner_tcp + cs_scanner_tcp_in_sp
    cs_cam_in_sp = cs_cam_in_scanner_tcp + cs_scanner_tcp_in_sp

    weight = cs_scanner_in_sp.location[0] / 350
    profile_local = geo.linear_profile_interpolation_sbs(profile_1, profile_2, weight)
    profile_local_data = profile_local.rasterize(20)
    num_raster_points = profile_local_data.shape[1]
    nhalf = int(num_raster_points / 2)
    ax_0.clear()
    ax_0.set_title("profile at x = " + str(np.round(cs_scanner_in_sp.location[0], decimals=1)) + "mm")
    ax_0.axis("auto")
    ax_0.plot(profile_local_data[0, 1:nhalf], profile_local_data[1, 1:nhalf], "b", label="ideal")
    ax_0.plot(profile_local_data[0, nhalf:], profile_local_data[1, nhalf:], "b")
    ax_0.plot(scanned_profiles_in_sp[idx, 2, :], scanned_profiles_in_sp[idx, 1, :], "r", label="scan")
    ax_0.set_xlim([-20, 20])
    ax_0.set_ylim([0, 12])
    ax_0.set_xlabel("z in mm")
    ax_0.set_ylabel("y in mm")
    ax_0.legend()
    ipvfig.scatters[0].sequence_index = idx.item() + idx_offset
    ipvfig.scatters[0].visible = True
    t_0 = scan_data.MH24_T01.data[idx]
    t_1 = scan_data.MH24_T02.data[idx]
    t_2 = scan_data.MH24_T03.data[idx]
    update_meshes(cs_scanner_in_sp, cs_torch_in_sp, cs_cam_in_sp, [0, 1, 0], t_0, t_1, t_2)


def update_output(time):
    if time < num_timesteps_scan_0:
        scn_idx = indices_scan_0[time]
        ipvfig.meshes[7].visible = False
        visualize_scan_phase(list_dsx_scan_data[0], scanned_profiles_layer_0_sp, scn_idx, num_timesteps_scan_0)
    elif time < num_timesteps_scan_0 + num_timesteps_movement:
        ipvfig.scatters[0].visible = False
        local_idx = time - num_timesteps_scan_0
        mvm_idx = indices_movement[local_idx]

        x_angles_base = machine_angles_degree_base[:, mvm_idx]
        x_coordinates_base = machine_coordinates_base[:, mvm_idx]
        x_orientation_base = rot.from_euler(angles=x_angles_base, seq="xyz", degrees=True).as_matrix()
        cs_x_in_base = tf.LocalCoordinateSystem(basis=x_orientation_base, origin=x_coordinates_base)
        cs_x_in_sp = cs_x_in_base - cs_ref_in_base - cs_sp_in_ref

        if mvm_idx > idx_switch_cs:
            cs_torch_in_sp = cs_torch_tcp_in_scanner_tcp + cs_x_in_sp
            cs_scanner_in_sp = cs_scanner_schematic + cs_x_in_sp
            cs_cam_in_sp = cs_cam_in_torch_tcp + cs_torch_in_sp
        else:
            cs_torch_in_sp = cs_x_in_sp
            cs_scanner_in_sp = cs_scanner_schematic + cs_scanner_tcp_in_torch_tcp + cs_torch_in_sp
            cs_cam_in_sp = cs_cam_in_torch_tcp + cs_torch_in_sp
        torch_color = [0, 1, 0]
        if dsx_relevant_machine_data.trigSchweissen.data[mvm_idx] > 0:
            brightness = (time % 5) / 4
            torch_color = [brightness, brightness, 1]
        t_0 = dsx_relevant_machine_data.MH24_T01.data[mvm_idx]
        t_1 = dsx_relevant_machine_data.MH24_T02.data[mvm_idx]
        t_2 = dsx_relevant_machine_data.MH24_T03.data[mvm_idx]
        update_meshes(cs_scanner_in_sp, cs_torch_in_sp, cs_cam_in_sp, torch_color, t_0, t_1, t_2)

        if local_idx < video_frames.shape[0]:
            ax_0.clear()
            ax_0.imshow(video_frames[time - num_timesteps_scan_0], cmap="gray", vmin=0, vmax=255)
            max_weld_idx = int(num_points_weld / 2)
            ipvfig.meshes[7].x = np.clip(data_weld[:, 0], 0, cs_torch_in_sp.location[0] + 10)
            ipvfig.meshes[7].visible = True

    else:
        scn_idx = indices_scan_1[time - num_timesteps_scan_0 - num_timesteps_movement]
        ipvfig.meshes[7].visible = True
        visualize_scan_phase(
            list_dsx_scan_data[1], scanned_profiles_layer_1_sp, scn_idx, num_timesteps_scan_1, num_scans_layer_0
        )


output = interactive_output(update_output, w)
upper_box.children = [info_box, ipvfig, out2]
lower_box.children = [play, time_slider]
display(box)
_ = ipv.view(elevation=10, azimuth=120, distance=1.7)